<a href="https://colab.research.google.com/github/sainikhila11/SaiNikhila_INFO5731_Spring2024/blob/main/Yavanamanda_Sai_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def movie_reviews(movie_id):
  #url for reviews page of the movie 'Barbie'
    url = f'https://www.imdb.com/title/tt1517268/reviews'
    #create list for reviews
    reviews = []
    try:
        while True:
          # request.get takes the request to the specified url and retreives to response
            response = requests.get(url)
            # extract html content from http response
            soup = BeautifulSoup(response.content, 'html.parser')

            # extract div elements from 'review contaainer' class
            review_containers = soup.find_all('div', class_='review-container')

            for container in review_containers:
                # Find the title within the container. this gives us the short summarized review of the text.
                title_element = container.find('a', class_='title')
                title = title_element.get_text(strip=True) if title_element else 'No Title'

                # Find the review text within the container. this gives us the complete review posted by user usng the text show-more__control
                review_text_element = container.find('div', class_='text show-more__control')
                review_text = review_text_element.get_text(strip=True) if review_text_element else 'No Review Text'

                # add retreived data to reviews list
                reviews.append((title, review_text))

            # conditional loop to retrieve reviews as the count we need is 1000
            if len(reviews) >= 1000 or not review_containers:
                break

            time.sleep(1)

    except Exception as e:
        print(f'Error occurred: {e}')

    return reviews[:1000]

movie_ids = ['tt1517268']  #id in imdb for barbie movie
all_reviews = [] #list to have reviews if multiple movies were taken

for movie_id in movie_ids:
    all_reviews.extend(movie_reviews(movie_id))
    if len(all_reviews) >= 1000:
        break

# Save the reviews to a CSV file
with open('imdb_reviews.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Review'])  # Header
    for title, review in all_reviews:
        writer.writerow([title, review])

# print number of reviews retrieved
print(f'Successfully saved {len(all_reviews)} reviews with titles.')

#download the csv file with reviews retrieved to local system
from google.colab import files
files.download('imdb_reviews.csv')

Successfully saved 1000 reviews with titles.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [9]:
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Remove noise (special characters and punctuations)
    # all characters are read one after one and only alphanumerics and spaces will be considered rest will be eliminated.
    cleaned_text = ''.join(char for char in text if char.isalnum() or char.isspace())

    # Remove numbers
    # all characters that are not numbers will be considered and moved to cleaned_text variable
    cleaned_text = ''.join(char for char in cleaned_text if not char.isdigit())

    # Lowercase all texts
    cleaned_text = cleaned_text.lower()

    return cleaned_text

def remove_stopwords(text):
    # load stop words and tokenize by using split function
    stop_words = set(stopwords.words('english'))
    words = text.split()
    # remove all stop words and move rest of the text to filtered_text
    filtered_text = ' '.join(word for word in words if word.lower() not in stop_words)

    return filtered_text

def stem_text(text):
  # Create an instance for Porter Stemmer algorithm
    ps = PorterStemmer()
    stemmed_text = ' '.join(ps.stem(word) for word in text.split())

    return stemmed_text

def lemmatize_text(text):
  # Create an instance of the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())

    return lemmatized_text

# Load the existing CSV file with scraped data
input_file_path = 'imdb_reviews.csv'
output_file_path = 'cleaned_imdb_reviews.csv'

# Read the existing reviews from the CSV file
all_reviews = []

with open(input_file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip header
    for row in reader:
        title, review = row
        all_reviews.append((title, review))

# Clean the reviews
cleaned_reviews = []

for title, review in all_reviews:
    cleaned_title = clean_text(title)
    cleaned_review = clean_text(review)


    cleaned_reviews.append((cleaned_title, cleaned_review))

# Save the cleaned reviews to a new CSV file
with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Cleaned Title', 'Cleaned Review'])  # Header
    for cleaned_title, cleaned_review in cleaned_reviews:
        writer.writerow([cleaned_title, cleaned_review])

# print number of reviews that have been cleaned
print(f'Successfully saved {len(cleaned_reviews)} cleaned reviews with titles in {output_file_path}.')

#download the csv file with reviews retrieved to local system
from google.colab import files
files.download('cleaned_imdb_reviews.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Successfully saved 1000 cleaned reviews with titles in cleaned_imdb_reviews.csv.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [10]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Read the cleaned reviews from the CSV file
cleaned_reviews = []

with open('cleaned_imdb_reviews.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip header
    for row in reader:
        cleaned_title, cleaned_review = row
        cleaned_reviews.append((cleaned_title, cleaned_review))

# Choose a sample review for analysis
sample_review = cleaned_reviews[0][1]  # Choose the review text from the first row

# (1) Parts of Speech (POS) Tagging
doc = nlp(sample_review)
pos_tags_count = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

for token in doc:
    if token.pos_ == 'NOUN':
        pos_tags_count['Noun'] += 1
    elif token.pos_ == 'VERB':
        pos_tags_count['Verb'] += 1
    elif token.pos_ == 'ADJ':
        pos_tags_count['Adjective'] += 1
    elif token.pos_ == 'ADV':
        pos_tags_count['Adverb'] += 1

print('POS Tags Count:', pos_tags_count)

# (2) Constituency Parsing and Dependency Parsing
# Print constituency parsing tree
print('\nConstituency Parsing Tree:')
for sent in doc.sents:
    for token in sent:
        print(f"{token.text} <--{token.dep_}-- {token.head.text}")

# (3) Named Entity Recognition (NER)
entities_count = {'PERSON': 0, 'ORG': 0, 'LOC': 0, 'PRODUCT': 0, 'DATE': 0}

for ent in doc.ents:
    entities_count[ent.label_] += 1

print('\nNamed Entity Recognition (NER) Count:')
for entity, count in entities_count.items():
    print(f"{entity}: {count}")


POS Tags Count: {'Noun': 30, 'Verb': 21, 'Adjective': 16, 'Adverb': 7}

Constituency Parsing Tree:
margot <--nsubj-- does
does <--ROOT-- does
the <--det-- best
best <--dobj-- does
with <--prep-- best
what <--dobj-- given
she <--nsubjpass-- given
s <--auxpass-- given
given <--pcomp-- with
but <--cc-- does
this <--det-- film
film <--nsubj-- was
was <--conj-- does
very <--advmod-- disappointing
disappointing <--acomp-- was
to <--prep-- disappointing
me <--pobj-- to
it <--nsubjpass-- marketed
was <--auxpass-- marketed
marketed <--ccomp-- was
as <--prep-- marketed
a <--det-- satire
fun <--amod-- satire
quirky <--amod-- satire
satire <--pobj-- as
with <--prep-- satire
homages <--pobj-- with
to <--prep-- marketed
other <--amod-- movies
movies <--pobj-- to
it <--nsubj-- started
started <--relcl-- movies
that <--det-- way
way <--npadvmod-- started
but <--cc-- marketed
ended <--conj-- marketed
with <--prep-- ended
overdramatized <--amod-- speeches
speeches <--pobj-- with
and <--cc-- speeches
an 

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

This assignemnt has been a little more helpful in terms of understanding web scraping with python better than the previous assignment. The previous assignment did work like an introduction to web scrape using python whereas this assignment helped in applying that to code. learning new libraries and functions that python provide to scrape and clean data has been very intresting.I did find a little trouble in giving the right input and right url to scrape in the question one, but looking through the HTML page source gave me the idea on which tags is having the data that i need. for example the 'div' tags has the fully written film revies under class 'text show-more__control' and the 'a' tags have the short version of reviews in class 'title'. Exploring through the source page to find the right elemts to tag in the code to perform scraping was something new that i had worked on. I do think the time provided is good enough for the assignment.